In [5]:
# The code was removed by Watson Studio for sharing.

In [1]:
!conda install -c conda-forge geocoder --yes

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20180914214529-0000
Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_0         3.5 MB  conda-forge
    certifi-2018.8.24          |           py35_1         138 KB  conda-forge
    orderedset-2.0             |           py35_0         685 KB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    ratelim-0.1.6              |           py35_0           5 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    ----------------------------------------------------------

In [4]:
# London lat lng
import geocoder

city = 'Bangkok'
# initialize your variable to None
coord = None
while(coord is None):
    g = geocoder.google(city)
    coord = g.latlng

lat, lng = coord

In [6]:
import requests

radius = 50000
LIMIT = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [7]:
results

{'meta': {'code': 200, 'requestId': '5b9c6512dd57976d89b2ea52'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4dfb6e666284607895334ba5-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_jazzclub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e7931735',
         'name': 'Jazz Club',
         'pluralName': 'Jazz Clubs',
         'primary': True,
         'shortName': 'Jazz Club'}],
       'id': '4dfb6e666284607895334ba5',
       'location': {'address': 'Samsen Rd.',
        'cc': 'TH',
        'city': 'พระนคร',
        'country': 'ประเทศไทย',
        'distance': 811,
        'formattedAddress': ['Samsen Rd.',
         'พระนคร',
         'กรุงเทพมหานคร 10200',
         'ประเทศไทย'],
        'labeledLatLngs': [{'label': 'display

In [17]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
df = json_normalize(venues) # flatten JSON
df = df[['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.id']]

df['venue.categories'] = df['venue.categories'].apply(lambda x : x[0]['name'])
df.columns = df.columns.map(lambda x : x.split('.')[-1])
df.head(2)

,name,categories,lat,lng,id
0,Adhere the 13th Blues Bar,Jazz Club,13.763043,100.498839,4dfb6e666284607895334ba5
1,Temple of the Emerald Buddha (วัดพระศรีรัตนศาส...,Buddhist Temple,13.751735,100.492651,4b0587f9f964a52001aa22e3
